In [5]:
!curl -O "https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  422k  100  422k    0     0   579k      0 --:--:-- --:--:-- --:--:--  579k


In [7]:
!unzip -o bank-additional.zip

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


2. We will then load it with Panda

In [1]:
import pandas as pd
data = pd.read_csv('bank-additional/bank-additional-full.csv')
print(data.shape)
data[:5]

(41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


3. Now let's upload the dataset to Amazon S3. We will use a default “Bucket” 
automatically created by SageMaker in the region we are running in.


In [2]:
import sagemaker
prefix = 'sagemaker/DEMO-smprocessing/input'
input_data = sagemaker.Session().upload_data(path='bank-additional/bank-additional-full.csv', key_prefix=prefix)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.sklearn.processing import SKLearnProcessor
role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
 role=role,
 instance_type='ml.m5.xlarge',
 instance_count=1)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
sklearn_processor.run(code='preprocessing.py',inputs=[ProcessingInput(
 source=input_data,
 destination='/opt/ml/processing/input')],
 outputs=[ProcessingOutput(output_name='train_data',
 source='/opt/ml/processing/train'),
 ProcessingOutput(output_name='test_data',
 source='/opt/ml/processing/test')]
 )


INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2023-12-17-15-45-03-373


...........................
/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
Reading input data from /opt/ml/processing/input/bank-additional-full.csv
Positive samples: 4639
Negative samples: 36537
Ratio: 7.88
Running preprocessing and feature engineering transformations
Train data shape after preprocessing: (32940, 58)
Test data shape after preprocessing: (8236, 58)
Saving training features to /opt/ml/processing/train/train_features.csv
Saving test features to /opt/ml/processing/test/test_features.csv
Saving training labels to /opt/ml/processing/train/train_labels.c

In [8]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()
output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])


s3://sagemaker-us-east-1-141154829940/sagemaker-scikit-learn-2023-12-17-15-45-03-373/output/train_data
s3://sagemaker-us-east-1-141154829940/sagemaker-scikit-learn-2023-12-17-15-45-03-373/output/test_data
